# 非极大值抑制应用(帧率有所降低)

In [ ]:
#导入oled屏幕库 Import oled screen library
import sys
sys.path.append('/home/pi/software/oled_yahboom/')
from yahboom_oled import *
# 创建oled对象 Create an oled object
oled = Yahboom_OLED(debug=False)
import numpy as np
import cv2
import os,time
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_utils
import ipywidgets.widgets as widgets
from image_fun import bgr8_to_jpeg

In [ ]:
# Init camera 
cap = cv2.VideoCapture(0)  # 定义摄像头对象，参数0表示第一个摄像头 Define the camera object, parameter 0 represents the first camera
cap.set(3, 320) # set Width
cap.set(4, 240) # set Height
cap.set(5, 30)  #设置帧率 Setting the frame rate
# cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
# cap.set(cv2.CAP_PROP_BRIGHTNESS, 60) #设置亮度 -64 - 64  0.0 Set Brightness -64 - 64 0.0
# cap.set(cv2.CAP_PROP_CONTRAST, 50) #设置对比度 -64 - 64  2.0 Set Contrast -64 - 64 2.0
# cap.set(cv2.CAP_PROP_EXPOSURE, 156)  #设置曝光值 1.0 - 5000  156.0 Set the exposure value 1.0 - 5000 156.0

# from picamera2 import Picamera2, Preview
# import libcamera
# picam2 = Picamera2()  
# camera_config = picam2.create_preview_configuration(main={"format":'RGB888',"size":(320,240)})
# camera_config["transform"] = libcamera.Transform(hflip=1, vflip=1)
# picam2.configure(camera_config) 
# picam2.start() 



In [ ]:
image_widget = widgets.Image(format='jpg', width=640, height=480)

In [ ]:
# Init tf model

MODEL_NAME = 'ssdlite_mobilenet_v2_coco_2018_05_09' #fast
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb' 
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt') 
NUM_CLASSES = 90 
IMAGE_SIZE = (12, 8) 
fileAlreadyExists = os.path.isfile(PATH_TO_CKPT) 

if not fileAlreadyExists:
    print('Model does not exsist !')
    exit

In [ ]:
# LOAD GRAPH
print('Loading...')
detection_graph = tf.Graph() 
with detection_graph.as_default(): 
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid: 
        serialized_graph = fid.read() 
        od_graph_def.ParseFromString(serialized_graph) 
        tf.import_graph_def(od_graph_def, name='')
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True) 
category_index = label_map_util.create_category_index(categories)
print('Finish Load Graph..')

In [ ]:
print(type(category_index))

In [ ]:
print("dict['Name']: ", category_index[1]['name'])

In [ ]:
# Main
oled.init_oled_process() #初始化oled进程 Initialize oled process
oled.add_line("OBJTYPE:", 1)
oled.add_line("None", 3)
oled.refresh()
t_start = time.time()
fps = 0
display(image_widget)
with detection_graph.as_default():
    with tf.compat.v1.Session(graph=detection_graph) as sess:
        while True:
            #frame = picam2.capture_array()
            ret, frame = cap.read()
#            frame = cv2.flip(frame, -1) # Flip camera vertically
#             frame = cv2.resize(frame,(320,240))
            ##############
            image_np_expanded = np.expand_dims(frame, axis=0) 
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0') 
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0') 
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0') 
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0') 
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

#             print('Running detection..') 
            (boxes, scores, classes, num) = sess.run( 
                [detection_boxes, detection_scores, detection_classes, num_detections], 
                feed_dict={image_tensor: image_np_expanded}) 

#             print('Done.  Visualizing..')
            # 应用非极大值抑制
            selected_indices = tf.image.non_max_suppression(
                np.squeeze(boxes),
                np.squeeze(scores),
                max_output_size=100,  # 根据需要设置最大输出数量
                iou_threshold=0.7)  # 调整重叠阈值
                
            # 使用 tf.gather 来高效地应用索引
            filtered_boxes = tf.gather(tf.squeeze(boxes), selected_indices)
            filtered_scores = tf.gather(tf.squeeze(scores), selected_indices)
            filtered_classes = tf.gather(tf.cast(tf.squeeze(classes), tf.int32), selected_indices)
            
            # 在会话中运行过滤操作
            (filtered_boxes, filtered_scores, filtered_classes) = sess.run([
                filtered_boxes,
                filtered_scores,
                filtered_classes
            ])
            
            # 可视化
            vis_utils.visualize_boxes_and_labels_on_image_array(
                frame,
                filtered_boxes,
                filtered_classes,
                filtered_scores,
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8)
            
            for i in range(0, 10):
                if scores[0][i] >= 0.7:
                    print(category_index[int(classes[0][i])]['name'])
                    oled.clear()
                    objtype_str=category_index[int(classes[0][i])]['name']
                    oled.add_line("OBJTYPE:", 1)
                    oled.add_line(objtype_str, 3)
                    oled.refresh()
            ##############
            fps = fps + 1
            mfps = fps / (time.time() - t_start)
            cv2.putText(frame, "FPS:" + str(int(mfps)), (10,25), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
            image_widget.value = bgr8_to_jpeg(frame)

            k = cv2.waitKey(1) & 0xff
            if k == 27:# press 'ESC' to quit
                cap.release()
                # 恢复屏幕基础数据显示 Restore basic data display on screen
                os.system("python3 /home/pi/software/oled_yahboom/yahboom_oled.py &")
                break

    

In [ ]:
cap.release()
# 恢复屏幕基础数据显示 Restore basic data display on screen
os.system("python3 /home/pi/software/oled_yahboom/yahboom_oled.py &")
# picam2.stop()
# picam2.close()
#最后需要释放掉摄像头的占用 Finally, you need to release the camera's occupancy